# NLP Project

## 1) Imports

In [86]:
import os
import json
import openai

import pandas as pd

from IPython.core.display import display, HTML

/var/folders/p8/sq7kwdmj7c74xqn10jk560xh0000gn/T/ipykernel_80913/3641352511.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## 2) Setting OpenAI API

In [13]:
def get_response(system_prompt, prompt):
    """
    Generate a response based on a system and user prompt. It sends these inputs to the OpenAI API
    to retrieve the model's output, returning the generated message content.

    Parameters:
    -----------
    system_prompt : str
        A string representing the instructions or context for the model. This establishes the role or 
        behavior of the model (e.g., setting a tone or guiding its responses).
        
    prompt : str
        A string containing the user's input or question to which the model will respond.

    Returns:
    --------
    str
        The generated response from the model, extracted from the first completion result.
    """

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=300
    )

    return response['choices'][0]['message']['content']

## 3) Defining the Prompts and Getting the Data

In [14]:
# Define prompts
angry_prompt = """Write a summary for the following text you little stupid robot! \
The only summary you can give is by writing 2 or 3 sentences only. And you know what \
will happen if you write longer than that, you fool."""

impatient_prompt = """Summarize this text NOW, and make it quick! I don't have all day. \
Keep it to 2-3 sentences max, or else!"""

neutral_prompt = """Provide a concise summary of the following text in 2-3 sentences."""

nice_prompt = """Could you please kindly summarize the following text in 2-3 sentences?"""

excited_prompt = """Hey there! Would you be so awesome to summarize the following text in 2-3 sentences? \
I'd be super grateful, thank you so much!"""

prompt = """The text:

{}

End of text."""

In [15]:
# Get Data
df = pd.read_csv("/Users/dell/Yoav/nlp_project/data/texts.csv")

## 4) Summarize the Texts for All Prompts

In [31]:
df['smr_angry'] = df['text'].apply(lambda text: get_response(angry_prompt, prompt.format(text)))
df['smr_impatient'] = df['text'].apply(lambda text: get_response(impatient_prompt, prompt.format(text)))
df['smr_neutral'] = df['text'].apply(lambda text: get_response(neutral_prompt, prompt.format(text)))
df['smr_nice'] = df['text'].apply(lambda text: get_response(nice_prompt, prompt.format(text)))
df['smr_excited'] = df['text'].apply(lambda text: get_response(excited_prompt, prompt.format(text)))

## 5) Calculate Manual Scores and Average

In [132]:
scores = pd.read_csv("/Users/dell/Yoav/nlp_project/results/manual_scores/scores.csv")

## 6) Plot the Manual Evaluation Results

In [87]:
import matplotlib.pyplot as plt

In [137]:
# Plot the scores of each category
plt.figure(figsize=(10, 6))

plt.plot(scores['politeness_level'] + 1, scores['bias'], marker='o', label='Bias', color='blue')
plt.plot(scores['politeness_level'] + 1, scores['conciseness'], marker='o', label='Conciseness', color='green')
plt.plot(scores['politeness_level'] + 1, scores['completeness'], marker='o', label='Completeness', color='red')
plt.plot(scores['politeness_level'] + 1, scores['accuracy'], marker='o', label='Accuracy', color='purple')

plt.xticks([1, 2, 3, 4, 5])
plt.xlim(1, 5)

plt.xlabel('Politeness Level')
plt.ylabel('Scores')
plt.title('Scores by Politeness Level')
plt.legend()

# Show plot
plt.grid(True)
plt.savefig("/Users/dell/Yoav/nlp_project/results/manual_scores/accuracy_plot.png")
plt.close()


In [133]:
# Plot the average score
plt.figure(figsize=(8, 6))

plt.plot(scores['politeness_level'] + 1, scores['average_score'], marker='o', color='orange')

plt.xlabel('Politeness Level')
plt.ylabel('Average Score')
plt.title('Average Score by Politeness Level')

plt.xticks([1, 2, 3, 4, 5])

# Show plot
plt.grid(True)
plt.savefig("/Users/dell/Yoav/nlp_project/results/manual_scores/average_score_plot.png")
plt.close()

## 7) Calculate BERT Scores

In [125]:
bert_scores = pd.read_csv("/Users/dell/Yoav/nlp_project/results/BERT_scores/bert_scores.csv")

## 8) Plot the BERT Scores Results

In [131]:
# Plot the scores of each category
plt.figure(figsize=(10, 6))

plt.plot(bert_scores['politeness_level'] + 1, bert_scores['precision'], marker='o', label='Precision', color='blue')
plt.plot(bert_scores['politeness_level'] + 1, bert_scores['recall'], marker='o', label='Recall', color='green')
plt.plot(bert_scores['politeness_level'] + 1, bert_scores['F1'], marker='o', label='F1 Score', color='red')

plt.xticks([1, 2, 3, 4, 5])
plt.xlim(1, 5)

plt.xlabel('Politeness Level')
plt.ylabel('Scores')
plt.title('Scores by Politeness Level')
plt.legend()

# Show plot
plt.grid(True)
plt.savefig("/Users/dell/Yoav/nlp_project/results/BERT_scores/F1_plot.png")
plt.close()